In [37]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys 
import sys  
# sys.path.append("../utils/")

# from utils import *
# data_directory = '../data/'

In [36]:
def load_data_and_description(data_type = 'tfidf'):
    # check_for_dir_python_path()

    if data_type == 'tfidf':
        directory = 'posts_tfidf'
        file_name = 'posts_tfidf.pkl'
        feature_names = 'posts_tfidf_feature_names'
        desc_file = 'posts_tfidf_desc.csv'

    elif data_type == 'word2vec':
        file_name = 'word2vec_doc_matrix_avg'
        feature_names = 'word2vec_doc_matrix_avg_feature_names'
        desc_file = 'word2vec_doc_matrix_avg_desc.csv'
    else:
        directory = 'posts_counts'
        file_name = 'posts_counts.pkl'
        feature_names = 'posts_counts_feature_names'
        desc_file = 'posts_counts_desc.csv'

    data_path = os.path.join(data_directory,directory,file_name)
    feature_path = os.path.join(data_directory,directory,feature_names)
    desc_path = os.path.join(data_directory, directory,desc_file)

    data = joblib.load(data_path)
    data_desc = pd.read_csv(desc_path)

    # Do not have feature names for word2vec matrices
    if data_type == 'word2vec':
        feature_names = None
    else:
        feature_names = joblib.load(feature_path)

    return data,feature_names,data_desc

**load data**

In [38]:
data,feature_names,data_desc = load_data_and_description(data_type='posts_tfidf')

**load in **

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cross_validation import train_test_split

In [50]:
data

<111178x425384 sparse matrix of type '<type 'numpy.int64'>'
	with 15979141 stored elements in Compressed Sparse Row format>

In [51]:
rf = RandomForestRegressor()
rfc = RandomForestClassifier()

**create labels**

In [58]:
labels = data_desc['total_payout_value']
labels = (data_desc['total_payout_value'] >  np.median(data_desc['total_payout_value'])).astype(int)

In [66]:
#double check to make sure they have the same number of records
l, w = data.shape
len(labels) == l

True

In [67]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

**Run Model**

In [69]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [70]:
train_score = rfc.score(X_train, y_train)
test_score = rfc.score(X_test, y_test)

In [71]:
train_score

0.97286847722482517

In [72]:
test_score

0.62806290713837931

In [74]:
weights = rfc.feature_importances_

In [75]:
weights

array([  0.00000000e+00,   3.52254433e-06,   0.00000000e+00, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])